Import stuff

In [8]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

#pv.set_jupyter_backend("ipyvtklink")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

Import Envelope Lattice

In [9]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/meshes/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
envelope_lattice.shape

env_cens = envelope_lattice.centroids_threshold(0)

print (env_cens)

[[-30. -90.  10.]
 [-30. -90.  20.]
 [-30. -90.  30.]
 [-30. -80.  10.]
 [-30. -80.  20.]
 [-30. -80.  30.]
 [-30. -70.  10.]
 [-30. -70.  20.]
 [-30. -70.  30.]
 [-30. -60.  10.]
 [-30. -60.  20.]
 [-30. -60.  30.]
 [-30. -50.  10.]
 [-30. -50.  20.]
 [-30. -50.  30.]
 [-30. -40.  10.]
 [-30. -40.  20.]
 [-30. -40.  30.]
 [-30. -30.  10.]
 [-30. -30.  20.]
 [-30. -30.  30.]
 [-30. -20.  10.]
 [-30. -20.  20.]
 [-30. -20.  30.]
 [-20. -90.  10.]
 [-20. -90.  20.]
 [-20. -90.  30.]
 [-20. -80.  10.]
 [-20. -80.  20.]
 [-20. -80.  30.]
 [-20. -70.  10.]
 [-20. -70.  20.]
 [-20. -70.  30.]
 [-20. -60.  10.]
 [-20. -60.  20.]
 [-20. -60.  30.]
 [-20. -50.  10.]
 [-20. -50.  20.]
 [-20. -50.  30.]
 [-20. -40.  10.]
 [-20. -40.  20.]
 [-20. -40.  30.]
 [-20. -30.  10.]
 [-20. -30.  20.]
 [-20. -30.  30.]
 [-20. -20.  10.]
 [-20. -20.  20.]
 [-20. -20.  30.]
 [-10. -90.  10.]
 [-10. -90.  20.]
 [-10. -90.  30.]
 [-10. -80.  10.]
 [-10. -80.  20.]
 [-10. -80.  30.]
 [-10. -70.  10.]
 [-10. -70

Calculate the border points (this calculates the border voxels of the envelope, not the points on the street)

In [10]:
border_lattice = envelope_lattice * 0.0
# border_lattice[:,:,1:-1] += 1.0
border_lattice[:,1:-1,:] += 1.0
border_lattice[1:-1,:,:] += 1.0
border_lattice = border_lattice != 2
border_lattice.centroids

cloud([[ -40., -100.,    0.],
       [ -40., -100.,   10.],
       [ -40., -100.,   20.],
       [ -40., -100.,   30.],
       [ -40., -100.,   40.],
       [ -40.,  -90.,    0.],
       [ -40.,  -90.,   10.],
       [ -40.,  -90.,   20.],
       [ -40.,  -90.,   30.],
       [ -40.,  -90.,   40.],
       [ -40.,  -80.,    0.],
       [ -40.,  -80.,   10.],
       [ -40.,  -80.,   20.],
       [ -40.,  -80.,   30.],
       [ -40.,  -80.,   40.],
       [ -40.,  -70.,    0.],
       [ -40.,  -70.,   10.],
       [ -40.,  -70.,   20.],
       [ -40.,  -70.,   30.],
       [ -40.,  -70.,   40.],
       [ -40.,  -60.,    0.],
       [ -40.,  -60.,   10.],
       [ -40.,  -60.,   20.],
       [ -40.,  -60.,   30.],
       [ -40.,  -60.,   40.],
       [ -40.,  -50.,    0.],
       [ -40.,  -50.,   10.],
       [ -40.,  -50.,   20.],
       [ -40.,  -50.,   30.],
       [ -40.,  -50.,   40.],
       [ -40.,  -40.,    0.],
       [ -40.,  -40.,   10.],
       [ -40.,  -40.,   20.],
       [ -

Plotting the code above

In [11]:
p = pv.Plotter(notebook=True)

base_lattice = border_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(247.21077512227743, 152.21077512227743, 232.21077512227743),
 (35.0, -60.0, 20.0),
 (0.0, 0.0, 1.0)]

Calculating the points on the street (defect)

In [12]:
#calculating maximum and minimum values

#minimum values for x and y
xmin, ymin, zmin = env_cens.min(axis=0)

#maximum values for x and y
xmax, ymax, zmax = env_cens.max(axis=0)

#change bound
xmin = xmin - 10
ymin = ymin - 10
xmax = xmax + 10
ymax = ymax + 10

#calculating the points on the street

#calculate the x coördinates
x = np.arange(xmin, xmax)

#calculate the y coördinates
y = np.arange(ymin, ymax)

X = np.tile(x, (2, 1))
X = np.append(X, np.tile(xmin, (len(y), 1)))
X = np.append(X, np.tile(xmax, (len(y), 1)))

Y = np.tile(ymin, (len(x), 1))
Y = np.append(Y, np.tile(ymax, (len(x), 1)))
Y = np.append(Y, np.tile(y, (2, 1)))

Z = np.zeros(len(X))

street_points = np.stack([X,Y,Z]).T
street_points = tg.cloud(street_points)
street_points = street_points.astype(int)
street_points


cloud([[ -40, -100,    0],
       [ -39, -100,    0],
       [ -38, -100,    0],
       ...,
       [ 120,  -13,    0],
       [ 120,  -12,    0],
       [ 120,  -11,    0]])

Visualizing

In [13]:
p = pv.Plotter(notebook=True)

street_points.fast_notebook_vis(p)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
#p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(257.0578654123585, 162.05786541235855, 239.55786541235855),
 (37.5, -57.5, 20.0),
 (0.0, 0.0, 1.0)]

Saving the streetpoints

In [14]:
street_points_path = os.path.relpath('../data/meshes/street_points.csv')
street_points.to_csv(street_points_path)